Pipeline
<ul><li>aka unicode</li>
    <li>Демин</li>
    <li>1feature-1word</li>
    <li>catboost</li>
    </ul>

In [196]:
import pandas as pd
import os.path
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [241]:
def convert_to_ascii(text):
    return " ".join(str(ord(char)) for char in text)

In [15]:
dataPath="data/"

In [134]:
table=pd.read_excel(os.path.join(dataPath,"1000 ФИО.xlsx"))

In [135]:
table.head(2)

,Фамилия,Имя,Отчество
0,Уварова,Виктория,Еремеевна
1,Белов,Бронислав,Якунович


In [242]:
# Формирование датасета
SurnameDf=pd.DataFrame(table["Фамилия"].values,columns=["word"])
NameDf=pd.DataFrame(table["Имя"].values,columns=["word"])
Patronymic=pd.DataFrame(table["Отчество"].values,columns=["word"])

SurnameDf["class"]=1
NameDf["class"]=2
Patronymic["class"]=3

data=pd.concat([SurnameDf,NameDf,Patronymic])

data["word"]=data["word"].astype(str)
data["word"]=data["word"].apply(lambda x: x.lower()) #Нижний регистр
data["lenth"]=data["word"].apply(lambda x: len(x)) #Длина слова

## Кодирование юникодом

In [ ]:
data["ascii_word"]=data["word"].apply(lambda x:convert_to_ascii(x) ) #Слово переведенное в ascii

In [181]:
data.head()

,word,class,lenth,ascii_word
0,уварова,1,7,1091 1074 1072 1088 1086 1074 1072
1,белов,1,5,1073 1077 1083 1086 1074
2,сергеева,1,8,1089 1077 1088 1075 1077 1077 1074 1072
3,фокин,1,5,1092 1086 1082 1080 1085
4,журавлёв,1,8,1078 1091 1088 1072 1074 1083 1105 1074


In [186]:
def convert_to_ascii2(text):
    return [ord(i) for i in text]

In [237]:
def words_to_dataFame(column,max_lenth_word):
    letter=['letter%s' % str(i) for i in range(1,max_lenth_word+1)]
    
    names_arr=[]
    for word in column.values:
        names_arr.append(convert_to_ascii2(word))
        
    df=pd.DataFrame(columns=letter)
    j=0
    for i in names_arr:
        df.loc[j]=i+(max_lenth_word-len(i))*[0]
        j+=1
    return df

In [243]:
m=np.max(data['lenth'].values)
letters=words_to_dataFame(data["word"],m)

In [252]:
letters.shape,data.shape

((3000, 15), (3000, 4))

In [250]:
data.head(2)

,word,class,lenth,ascii_word
0,уварова,1,7,1091 1074 1072 1088 1086 1074 1072
1,белов,1,5,1073 1077 1083 1086 1074


In [251]:
letters.head(2)

,letter1,letter2,letter3,letter4,letter5,letter6,letter7,letter8,letter9,letter10,letter11,letter12,letter13,letter14,letter15
0,1091,1074,1072,1088,1086,1074,1072,0,0,0,0,0,0,0,0
1,1073,1077,1083,1086,1074,0,0,0,0,0,0,0,0,0,0


In [286]:
df=pd.concat([letters[letters.columns],data[['lenth','class']]],axis=1, join_axes=[data.index])

In [287]:
df.shape

(3000, 17)

In [288]:
df.head()

,letter1,letter2,letter3,letter4,letter5,letter6,letter7,letter8,letter9,letter10,letter11,letter12,letter13,letter14,letter15,lenth,class
0,1091,1074,1072,1088,1086,1074,1072,0,0,0,0,0,0,0,0,7,1
1,1073,1077,1083,1086,1074,0,0,0,0,0,0,0,0,0,0,5,1
2,1089,1077,1088,1075,1077,1077,1074,1072,0,0,0,0,0,0,0,8,1
3,1092,1086,1082,1080,1085,0,0,0,0,0,0,0,0,0,0,5,1
4,1078,1091,1088,1072,1074,1083,1105,1074,0,0,0,0,0,0,0,8,1


In [293]:
X,y=df.iloc[:,:-1],df["class"]

In [297]:
# Качество модели
rf=RandomForestClassifier(random_state=42)
scorrs=cross_val_score(rf,X,y,cv=5)
np.mean(scorrs)

0.79866666666666675

## bag of letters